In [1]:
!pip install lxml

In [1]:
from bs4 import BeautifulSoup
import requests
import re, os
import random

import pandas as pd

from tqdm import tqdm

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36"
}

In [3]:
def get_page_url(page_nr=0):
    return f"https://www.shorpy.com/node?page={page_nr}"

In [4]:
def resolve_year(title):
    pattern = r"(?P<YEAR>\d\d\d\d)"
    m = re.findall(pattern, title)
    if len(m) == 0:
        return None
    return m[-1]

In [5]:
def download_image(url, filename):
    try:
        if os.path.exists(filename):
            return True
        response = requests.get(url)
        with open(filename, "wb+") as f:
            f.write(response.content)
        return True
    except:
        return False

In [6]:
def download_pages():
    page_nr = 0
    img_counter = 0
    try:
        data = []
        except_counter = 0
        
        for page_nr in tqdm(range(0, 1044)):
            page_url = get_page_url(page_nr)
            page = requests.get(page_url, headers=headers)
            soup = BeautifulSoup(page.text, "lxml")
            for content in soup.select("div.node > div.content"):
                try:
                    img_counter += 1
                    img = content.select("a > img")[0]
                    p = content.select("p")[0]
                    src = img["src"]
                    title = img["title"]
                    year = int(resolve_year(title))
                    text = p.text[:-16]
                    downloaded = download_image(src, f"images/{img_counter}_{year}.jpg")
                    data.append((year, src, text, downloaded))
                except Exception as e:
                    except_counter += 1  
        
    except Exception as e:
        print(e)
        print("Stopped at", page_nr)
    
    finally:
        df = pd.DataFrame(data, columns=["year", "src", "text", "downloaded"])
        df.to_csv("info.csv", index=False)
        return df

In [ ]:
df = download_pages()

 13%|█▎        | 135/1044 [31:44<3:57:31, 15.68s/it]

In [24]:
df.iloc[0]["src"]

'https://www.shorpy.com/files/images/2336_SHORPY.preview.jpg'

In [ ]:
df = pd.read_csv("info.csv")

In [ ]:
df["downloaded"] = False

In [ ]:
srcs = zip(list(range(0, df.shape[0])), list(df["src"]), list(df["year"]))

In [ ]:
srcs = list(srcs)

In [ ]:
counter = 0

In [ ]:
for (i, src, year) in tqdm(srcs):
    try:
        counter += 1
        filename = f"images/{counter}_{year}.jpg"
        if os.path.exists(filename):
            continue
        response = requests.get(src)
        with open(filename, "wb+") as f:
            f.write(response.content)
        df.loc[i, "downloaded"] = True
    except Exception as e:
        print(e)
    finally:
        df.to_csv("info.csv", index=False)

In [ ]:
df["downloaded"].sum()

In [26]:
download_image("https://www.shorpy.com/files/images/2336_SHORPY.preview.jpg", "images/test")

False

In [28]:
response = requests.get("https://www.shorpy.com/files/images/2336_SHORPY.preview.jpg")
with open("images/test.jpg", "wb+") as f:
    f.write(response.content)